# 🔧 Project: Optimizing Windows Audit Policies for Efficient Splunk Detection

## 🎯 Project Goal

This project aims to redesign Windows audit policy configurations to reduce log noise and enhance detection clarity within Splunk. The objective is to produce a streamlined, purposeful audit policy that supports effective threat detection, baselining, and investigative workflows in a lab or SOC simulation environment. The work is driven by operational findings from previous log analysis and seeks to align audit settings with actual detection value.

---

## ✅ Project Outline

### Phase 1 – Identify High-Value vs. High-Noise Event IDs

**Objective:**  
Establish which Windows Security Event IDs are valuable for detection and which contribute excessive noise.

**Tasks:**
- Review key event categories (e.g., Logon, Process Tracking, Object Access)
- Identify high-volume event codes from previous log data
- Map each to its detection value (e.g., 4624 = useful, 4689 = low value)
- Output: Table of event codes with actionability ratings

---

### Phase 2 – Define Audit Policy Goals for SOC Use

**Objective:**  
Align audit configuration with detection priorities based on known attacker behavior and SOC monitoring needs.

**Tasks:**
- Identify detection goals (e.g., brute force attempts, account escalation)
- Map those goals to specific audit subcategories and event codes
- Output: Rationale for including or excluding each audit setting

---

### Phase 3 – Redesign the Audit Policy Configuration

**Objective:**  
Develop a new audit policy that balances detection power with signal quality and manageable log volume.

**Tasks:**
- Compare current policy with proposed changes
- Justify changes per event category and operational need
- Output: Recommended `auditpol` or GPO configuration

---

### Phase 4 – Apply and Verify the New Policy

**Objective:**  
Implement the optimized configuration and verify that intended events are still being captured.

**Tasks:**
- Apply new audit settings using `auditpol.exe`
- Perform controlled test activity to generate logs
- Use SPL queries to confirm that key events are being logged
- Output: Verification results and query output summaries

---

### Phase 5 – Documentation and Final Recommendations

**Objective:**  
Summarize outcomes and provide guidance for future Splunk analysis or policy refinement.

**Tasks:**
- Document which audit changes led to meaningful improvements
- Identify any remaining noise or data gaps
- Output: Final recommendation for lab or production use


## Phase 1 – Review and Justify Audit Policy Reconfiguration

### Objective  
Summarize findings from the previous project to identify the problem.  
Describe high-volume events (e.g., 4688, 4689, 4703) and why they overwhelmed analysis.  
Clarify which logs were useful to detection (e.g., 4624, 4625, 4672, 4720, 4728) and why they matter.  
Justify why an optimized audit policy is needed for lab and SOC simulation purposes.

---

### Summary of Findings from Baseline Project

The previous Splunk-based project revealed a key operational challenge: the volume and frequency of Windows Security events made it difficult to isolate suspicious activity from routine background processes. While useful security data was present, it was buried in a flood of high-verbosity logs that hindered rapid triage and detection.

A significant portion of the dataset consisted of low-value, high-frequency events that were not critical to the project's primary objective of identifying indicators of compromise (IOCs). Without tuning, the system's default audit configuration overwhelmed both visual analysis and SPL-based filtering with excessive noise.

---

### High-Volume Event Codes That Overwhelmed Analysis

The following event codes were among the most frequent and contributed heavily to log volume:

| Event ID | Description                                | Frequency | Analysis Impact |
|----------|--------------------------------------------|-----------|-----------------|
| 4688     | Process creation                           | Very High | Excessive volume made pattern recognition difficult without heavy filtering. |
| 4689     | Process termination                        | High      | Paired with 4688, but added little value to IOC discovery. |
| 4703     | User rights assignment change              | Medium    | Frequent updates by system services produced noise irrelevant to detection. |

These events, while useful in detailed forensic investigations, introduced significant overhead when the goal was efficient anomaly detection in a lab or SOC training environment.

---

### Event Codes That Were Useful for Detection

The following event codes were instrumental in identifying suspicious behavior and should remain enabled in an optimized policy:

| Event ID | Description                                | Value to Detection |
|----------|--------------------------------------------|--------------------|
| 4624     | Successful logon                           | Core signal for user activity baselining and off-hours logon analysis. |
| 4625     | Failed logon                               | Key indicator for brute-force attempts and authentication failures. |
| 4672     | Special privileges assigned at logon       | Identifies privileged user sessions, especially outside normal hours. |
| 4720     | User account creation                      | Critical for detecting unauthorized local account creation. |
| 4728     | Member added to security-enabled group     | Reveals potential privilege escalation attempts. |

These events directly supported multiple detection objectives, including brute force analysis, privilege abuse, and group membership changes. Their signal quality was high, and their frequency was low enough to support visibility without overwhelming the dataset.

---

### Justification for Audit Policy Optimization

The default Windows audit configuration generates high volumes of telemetry that can obscure the security signal. For lab environments, SOC simulations, and focused detection exercises, a more selective policy is needed.

Optimizing the audit policy involves:

- **Reducing noise** from routine process and rights-assignment logs that lack clear IOC value.
- **Retaining high-signal events** relevant to authentication, privilege use, and account modification.
- **Improving analyst focus** by simplifying the event landscape during triage.

The next phase will evaluate which event categories can be safely disabled or reduced in scope to improve efficiency while maintaining detection integrity.



### Phase 2 – Define Audit Policy Goals for SOC Use

**Objective:**  
Align audit configuration with detection priorities based on known attacker behavior and SOC monitoring needs.

---

#### 1. Detection Goals and SOC Priorities

Audit policy configuration should support early threat detection, lateral movement tracking, and escalation identification. In a SOC environment, common detection goals include:

- **Brute force attacks**  
  Requires visibility into failed logons (Event ID 4625) and possibly account lockouts (4740).
  
- **Unauthorized account creation or privilege escalation**  
  Requires auditing of account management and group membership events (e.g., 4720, 4728, 4732, 4756).

- **Use of privileged accounts**  
  Requires tracking successful logons (4624) and assignment of special privileges (4672).

- **Execution of suspicious processes**  
  Requires detailed process tracking (e.g., 4688), command-line auditing, and PowerShell usage visibility.

- **Persistence mechanisms**  
  Requires auditing of scheduled task creation (4698), service installation (4697), and registry modifications.

---

#### 2. Mapping Goals to Audit Subcategories and Event Codes

| Detection Objective                      | Recommended Subcategory             | Example Event IDs        | Rationale                                                                 |
|-----------------------------------------|-------------------------------------|---------------------------|---------------------------------------------------------------------------|
| Failed logons / brute force             | Logon/Logoff                        | 4625, 4771, 4776          | Enables tracking of repeated failed authentication attempts              |
| 
Privilege use / escalation              | Logon/Logoff                        | 4672                      | Detects assignment of admin-level privileges at logon               |
| 
Account creation / management           | Account Management                  | 4720, 4722, 4726          | Tracks new account creation, disabling, and deletion                     |
| 
Group membership changes                | Account Management                  | 4728, 4729, 4732, 4756    | Reveals elevation by group manipulation                                  |
| 
Successful interactive or remote logon  | Logon/Logoff                        | 4624                      | Tracks all logon attempts; combined with logon type for context          |
| 
Process creation                        | Detailed Tracking                   | 4688, 4689                | Shows execution chains; needed for behavioral analysis                   |
| 
Scheduled task creation / service mods  | Object Access / System              | 4697, 4698                | Indicates persistence via service/task creation                          |
| 
Network activity visibility             | Object Access / Filtering Platform  | 5156, 5447, 5449          | Provides low-level insight into outbound and inbound network behavior    |

---

#### 3. Output: Audit Configuration Rationale

Below is a summary of which subcategories should be **enabled** and **why**, based on detection goals:

- **Logon/Logoff**:  
  Crucial for detecting brute force, credential abuse, and interactive access.  
  Enable: Success and Failure

- **Account Management**:  
  Needed for detection of user creation, deletion, group changes, and privilege escalation.  
  Enable: Success

- **Detailed Tracking**:  
  Enables process creation logging (Event 4688), which is essential for detecting LOLBins, malware, and attacker tooling.  
  Enable: Success (Failure not applicable)

- **Object Access**:  
  Required to detect registry/service manipulation or scheduled task creation. Must also enable specific object types.  
  Enable: Success and Failure (as needed by config)

- **System**:  
  Useful for detecting changes to system services and audit policy tampering.  
  Enable: Success

- **DS Access**:  
  Not relevant in non-domain environments; disable unless part of domain controller.  
  Disable

- **Policy Change**:  
  Enables detection of changes to audit policy and user rights assignment.  
  Enable: Success

- **Privilege Use**:  
  Tracks use of admin rights or sensitive privileges.  
  Enable: Success and Failure

---

This phase ensures that audit policies align with known attacker techniques (e.g., MITRE ATT&CK) and common SOC detection workflows. The result is a focused audit configuration that prioritizes high-value events while reducing noise.


## Phase 3 – Redesign the Audit Policy Configuration

### Objective
Develop and justify a new Windows audit policy that optimizes detection value, reduces noise, and enables effective SOC or lab analysis in Splunk.

---

### 1. Compare Current Policy to Proposed Changes

**Current Policy (Based on Initial Project):**
- **Very verbose:** Nearly all auditing subcategories enabled, especially process tracking, logon/logoff, and object access.
- **Result:**
    - Excessive event volume: 65,000+ Security events in hours of uptime.
    - High-frequency, low-value events:
        - *4688* (Process creation) and *4689* (Process exit) dominate logs but obscure actionable signals.
        - *4703* (User rights assignment changes) creates additional noise.

**Findings:**
- Most event volume is background/system-driven, not analyst-initiated or attacker-driven.
- High-value detection signals (successful/failed logons, privilege escalation, account creation, group membership changes) are present but hard to triage due to noise.

---

### 2. Justification for Policy Changes

| Event Category         | Previous Setting      | Problem Identified             | Proposed Change      | Rationale                                                               |
|-----------------------|----------------------|-------------------------------|----------------------|-------------------------------------------------------------------------|
| **Logon/Logoff**      | Success & Failure    | Needed for detection           | **Keep: S & F**      | Essential for authentication, brute force, off-hours access, escalation |
| **Account Management**| Success & Failure    | Needed for detection           | **Keep: Success**    | Captures user/group/account changes and escalation                      |
| **Detailed Tracking** | Success              | 4688: Too noisy at default     | **Tune/Restrict**    | Enable for targeted process types or with command-line logging only      |
| **Object Access**     | Not tuned            | Noise from registry, services  | **Selective**        | Enable only for scheduled task, service install, registry key of interest|
| **System**            | Success              | Some value, some noise         | **Keep: Success**    | Detects policy tampering, critical system changes                       |
| **Policy Change**     | Success              | Valuable                       | **Keep: Success**    | Needed to detect audit policy changes, user rights assignment           |
| **Privilege Use**     | Success & Failure    | Some value, low frequency      | **Keep: S & F**      | Required for escalation and privilege abuse detection                   |
| **DS Access**         | Disabled/Not needed  | Not relevant                   | **Disable**          | Not applicable in standalone/lab hosts                                  |
| **Filtering Platform**| Enabled              | Moderate/noisy                 | **Review**           | Enable if network filtering is required for the detection use case      |

**Summary of Major Changes:**
- **Reduce process tracking verbosity** (4688/4689) unless command-line is required for behavioral detection or unless investigating specific attack chains.
- **Focus object access auditing** to high-value items only (services, scheduled tasks, select registry keys).
- **Keep authentication and account management signals fully enabled.**
- **Disable Directory Service auditing in standalone environments.**
- **Tune filtering platform logs based on lab goals (can be noisy if left broad).**

---

### 3. Recommended Audit Policy Settings

Below is a sample configuration using `auditpol.exe` commands to implement these recommendations. (Customize as needed for your environment.)

```shell
:: Enable critical authentication auditing
auditpol /set /category:"Logon/Logoff" /success:enable /failure:enable

:: Enable account management events (success only)
auditpol /set /category:"Account Management" /success:enable /failure:disable

:: Tune process creation tracking (keep only as needed)
auditpol /set /subcategory:"Process Creation" /success:enable
:: Optionally, disable process termination if not needed:
auditpol /set /subcategory:"Process Termination" /success:disable

:: Restrict object access auditing to high-value objects via SACLs or enable for scheduled task/service registry keys as needed

:: Enable system auditing (success)
auditpol /set /category:"System" /success:enable /failure:disable

:: Enable policy change auditing
auditpol /set /category:"Policy Change" /success:enable /failure:disable

:: Enable privilege use auditing (success & failure)
auditpol /set /category:"Privilege Use" /success:enable /failure:enable

:: Disable DS Access (unless domain controller)
auditpol /set /category:"DS Access" /success:disable /failure:disable

:: Review Filtering Platform logs (enable only if monitoring network connections)
:: For example:
auditpol /set /subcategory:"Filtering Platform Connection" /success:enable /failure:disable

Note: For granular object access (services, scheduled tasks, critical registry keys), configure SACLs via object properties—not just global auditpol settings.

4. Rationale & Expected Outcome
Noise Reduction:

Lower log volume by disabling low-value, high-frequency event sources (especially process exits and non-critical object access).

Detection Integrity:

All essential signals for credential misuse, privilege escalation, account changes, and persistence are retained.

Efficiency:

Enables faster triage and higher analyst confidence in both lab and SOC simulation settings.

Adaptability:

Policy can be further tuned for new detection use cases or when event volume analysis shows new patterns.

5. Next Steps
Apply these settings on the lab/test system using auditpol.exe or via GPO.

Perform test activity (e.g., logon attempts, group membership changes, process launches).

Validate that key detection events are still being logged, and volume is reduced compared to baseline.






 **Phase 4 – Apply and Verify the New Policy**

**Objective**
Implement the optimized Windows audit policy configuration and verify that high-value security events are still captured, while overall log noise is reduced.

---

 **1. Apply the Optimized Audit Policy**

Use the `auditpol.exe` commands (from Phase 3) or equivalent Group Policy settings to update the audit configuration on your lab or test system.



```shell
:: Apply recommended settings (adjust as needed for your environment)
auditpol /set /category:"Logon/Logoff" /success:enable /failure:enable
auditpol /set /category:"Account Management" /success:enable /failure:disable
auditpol /set /subcategory:"Process Creation" /success:enable
auditpol /set /subcategory:"Process Termination" /success:disable
auditpol /set /category:"System" /success:enable /failure:disable
auditpol /set /category:"Policy Change" /success:enable /failure:disable
auditpol /set /category:"Privilege Use" /success:enable /failure:enable
auditpol /set /category:"DS Access" /success:disable /failure:disable
auditpol /set /subcategory:"Filtering Platform Connection" /success:enable /failure:disable

Note:
For object access to specific resources (services, scheduled tasks, registry keys), configure SACLs via object properties, not only via auditpol.

**2. Generate Controlled Test Activity**

**Purpose:**  
Create specific, realistic security events on the Windows system so we can later verify (in Step 3) that our optimized audit policy is capturing all critical detection scenarios.

** Preparation**

- Ensure you are logged in with an account that has local administrator rights.
- **Create a second, non-administrator user account for testing:**
    1. Press `Windows + X` and select **Computer Management**.
    2. In the left pane, go to **Local Users and Groups** > **Users**.
    3. Right-click **Users** and select **New User…**.
    4. Enter a username (e.g., `testuser`) and a password. Uncheck "User must change password at next logon" for convenience. Click **Create**.
    5. Close the New User dialog. The `testuser` account is now available for testing.
- You will use both your admin account and the new `testuser` account in the steps below.

---

 A. Interactive Logon (Local Login)

1. **Lock your screen** (`Windows` + `L`), then unlock by entering your password (admin account).
2. **Switch User:** At the lock screen, select **Switch User**.
3. **Log in as `testuser`** using the credentials you just created.

---

#### B. Remote Logon (Optional, if system is configured for RDP)

1. If Remote Desktop is enabled, connect to the machine using Remote Desktop Connection (`mstsc.exe`) from another computer.
2. Log in as either your admin account or `testuser`.

---

#### C. Failed Logon Attempt

1. At the Windows logon screen, select `testuser`.
2. Enter an incorrect password two or three times to simulate failed logon attempts (this triggers 4625 events).

---

#### D. Account Creation

1. **(Already completed in Preparation step)**  
   - No action needed unless you want to create an additional test account.

---

#### E. Group Membership Change

1. In **Computer Management** > **Local Users and Groups** > **Users**, right-click `testuser` and select **Properties**.
2. Go to the **Member Of** tab.
3. Click **Add**, type `Administrators`, and click **OK**.
4. Apply and close.

---

#### F. Process Launch

1. **Open Command Prompt:**
    - Press `Windows + R`, type `cmd`, press Enter.
    - Type `exit` to close.
2. **Open PowerShell:**
    - Press `Windows + R`, type `powershell`, press Enter.
    - Type `exit` to close.
3. **Launch Python (if installed):**
    - Press `Windows + R`, type `cmd`, press Enter.
    - Type `python` and press Enter, then type `exit()` and press Enter.

---

#### G. Modify Scheduled Task

1. **Open Task Scheduler:**
    - Press `Windows + R`, type `taskschd.msc`, press Enter.
2. Click **Create Basic Task…**
3. Name it (e.g., `SplunkAuditTest`), set any simple trigger and action, and finish the wizard.
4. Delete the test task after completion.

---

#### H. Modify a Windows Service (Optional, Advanced)

1. Open **Command Prompt as Administrator**.
2. Create a dummy service for test purposes (if desired), or change the startup type of an existing service:
    - Example:  
      `sc config wuauserv start= disabled`
    - (This changes the Windows Update service to "Disabled"—reverse after testing.)

---

#### Checklist After Completing All Actions

- Wait a few minutes for all events to be written to the Security log.
- Note the exact time you performed each action for easier searching in Splunk later.

---



**3. Verify Logging in Splunk**
Use Splunk SPL queries to confirm that the optimized policy is capturing all critical events and that log volume is reduced compared to baseline.

Example SPL Queries:


index=main EventCode IN (4624,4625,4672,4688,4720,4728,4698)
| stats count by EventCode, Account_Name, host

Check that each test action above generates the expected event.

Compare event counts for high-volume types (e.g., 4688) before and after policy optimization.

4. Output: Verification Results and Query Summaries
Sample Verification Table:

Event Code	Detection Objective	Event Observed After Change?	Comments
4624	Interactive logon	Yes/No	
4625	Failed logon	Yes/No	
4672	Privileged logon	Yes/No	
4688	Process creation	Yes/No	Volume reduced?
4720	Account creation	Yes/No	
4728	Group membership change	Yes/No	
4698	Scheduled task creation	Yes/No	

Document any missing events or issues.

Note improvements in analyst focus and log clarity.

Record any remaining noise (unexpected high-frequency events).

5. Interpretation and Next Steps
If all key events are observed and overall event volume is reduced, the policy is successful.

If critical events are missing, adjust the audit policy and re-test.

If noise persists from specific event types, consider further refinement or targeted SACL changes.

Phase 4 Complete:
The optimized audit policy is now implemented, tested, and validated. This forms the new baseline for efficient, detection-focused analysis in your SOC or lab Splunk environment.




 **Phase 5 – Documentation and Final Recommendations**

**Objective:**  
Summarize outcomes and provide guidance for future Splunk analysis or policy refinement.

---

 **1. Document Which Audit Changes Led to Meaningful Improvements**

The most impactful changes were:
- **Disabling high-noise events** such as Process Termination (4689), which dramatically reduced overall log volume and simplified event review.
- **Retaining only high-value audit categories** (Logon/Logoff, Account Management, Detailed Tracking for Process Creation, Privilege Use), ensuring all critical detection scenarios (logon, failed logon, privilege escalation, process creation, account changes) remained visible in Splunk.
- **Adding network connection auditing** (Filtering Platform Connection), if enabled, for enhanced visibility of lateral movement attempts.

This resulted in a **much cleaner event stream**—making it far easier to identify meaningful security activity while minimizing analysis fatigue.

---

 **2. Identify Any Remaining Noise or Data Gaps**

- **Remaining Noise:**  
  Routine system and Splunk service activity still generates some 4688 (Process Creation) and 4624 (Service Logon) events, but the overall volume is manageable and expected for a baseline system.
- **Data Gaps:**  
  All key detection signals tested (logon successes and failures, group membership changes, process launches, scheduled task creation) were successfully captured and visible in Splunk. No significant detection gaps were observed in this lab scenario.
- **Caveat:**  
  If future detection requires detailed monitoring of specific files, services, or registry keys, targeted Object Access auditing (SACLs) may be needed—currently out of scope.

---

 **3. Output: Final Recommendation for Lab or Production Use**

- **This optimized audit policy provides a strong foundation** for SOC or lab-based detection workflows, balancing visibility and log volume for efficient Splunk analysis.
- Further improvements could include periodic policy review, tailoring process tracking to known risk areas, or adding object access auditing only for highly sensitive assets.
- For most environments, these settings should serve as a practical baseline. Adjustments can be made as real-world use uncovers new detection needs or sources of unnecessary noise.

---

*End of Project: The system is now configured for efficient, detection-focused log collection and analysis in Splunk. Future refinements should be based on ongoing monitoring and evolving detection priorities.*
